In [51]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
from alibi.datasets import fetch_adult
from alibi.models.tensorflow import HeAE
from alibi.models.tensorflow import Actor, Critic
from alibi.models.tensorflow import ADULTEncoder, ADULTDecoder
from alibi.explainers.cfrl_base import Callback
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, get_statistics, \
    get_conditional_vector, apply_category_mapping
import random

def set_seed(s=0):
    random.seed(s)
    np.random.seed(s)
    tf.random.set_seed(s)






In [52]:
import logging
from typing import Optional, Tuple, Union
from sklearn.preprocessing import LabelEncoder
from alibi.utils.data import Bunch

logger = logging.getLogger(__name__)

df = pd.read_csv('.\\pre_bank.csv')
categorical_features = [
    'job',
    'marital',
    'education',
    'default',
    'housing',
    'loan',
    'contact',
    'month',
    'poutcome'
]
df[categorical_features] = df[categorical_features].astype(str)


def fetch_bank(
    features_drop: Optional[list] = None, 
    return_X_y: bool = False, 
    categorical_features: Optional[list] = None
) -> Union[Bunch, Tuple[np.ndarray, np.ndarray]]:
    
    raw_data = df
    # get labels, features and drop unnecessary features
    labels = (raw_data['y'] == 1).astype(int).values

    if features_drop is None:
        features_drop = []
    features_drop = features_drop + ['y']

    data = raw_data.drop(features_drop, axis=1)
    features = list(data.columns)

    # 关键部分：外部指定优先，否则自动推断
    if categorical_features is None:
        categorical_features = [f for f in features if data[f].dtype == 'O']

    category_map = {}
    for f in categorical_features:
        le = LabelEncoder()
        data_tmp = le.fit_transform(data[f].values)
        data[f] = data_tmp
        category_map[features.index(f)] = list(le.classes_)

    # only return data values
    data = data.values
    target_names = [0, 1]

    if return_X_y:
        return data, labels

    return Bunch(
        data=data, 
        target=labels, 
        feature_names=features, 
        target_names=target_names, 
        category_map=category_map
    )

bank = fetch_bank(
    categorical_features=categorical_features
)


In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# 1️⃣ 划分类别型和数值型特征


# Separate columns in numerical and categorical.
categorical_names = [bank.feature_names[i] for i in bank.category_map.keys()]
categorical_ids = list(bank.category_map.keys())

numerical_names = [name for i, name in enumerate(bank.feature_names) if i not in bank.category_map.keys()]
numerical_ids = [i for i in range(len(bank.feature_names)) if i not in bank.category_map.keys()]

# Split data into train and test
X, Y = bank.data, bank.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [54]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
# Define numerical standard scaler.
num_transf = MinMaxScaler()

# Define categorical one-hot encoder.
cat_transf = OneHotEncoder(
    categories=[range(len(x)) for x in bank.category_map.values()],
    handle_unknown="ignore"
)

# Define column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transf, numerical_ids),
        ("cat", cat_transf, categorical_ids),
    ],
    sparse_threshold=0
)

In [55]:
# Fit preprocessor.
preprocessor.fit(X_train)

# Preprocess train and test dataset.
X_train_ohe = preprocessor.transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

In [56]:
from tensorflow import keras
def build_simple_dnn():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(32, activation='relu', input_shape=(49,)))  # 输入31维特征
    model.add(keras.layers.Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
model = build_simple_dnn()
model.load_weights('my_model_weights_bank.h5')   # 加载之前保存的权重

In [57]:
# Define prediction function.
predictor = lambda x: model.predict(preprocessor.transform(x), verbose=0)

predictor(X_test[0:2])

array([[0.08495279, 0.91504717],
       [0.02794113, 0.97205895]], dtype=float32)

In [58]:
from sklearn.metrics import accuracy_score
# Compute accuracy.
acc = accuracy_score(y_true=Y_test, y_pred=predictor(X_test).argmax(axis=1))
print("Accuracy: %.3f" % acc)

Accuracy: 0.854


In [59]:
numerical_names

['age', 'balance', 'day', 'duration', 'campaign']

In [60]:
# Define attribute types, required for datatype conversion.
feature_types = {'age': int, 'balance': int, 'day': int, 'duration': int, 'campaign': int}

# Define data preprocessor and inverse preprocessor. The invers preprocessor include datatype conversions.
heae_preprocessor, heae_inv_preprocessor = get_he_preprocessor(X=X_train,
                                                               feature_names=bank.feature_names,
                                                               category_map=bank.category_map,
                                                               feature_types=feature_types)

# Define trainset
trainset_input = heae_preprocessor(X_train).astype(np.float32)
trainset_outputs = {
    "output_1": trainset_input[:, :len(numerical_ids)]
}

for i, cat_id in enumerate(categorical_ids):
    trainset_outputs.update({
        f"output_{i+2}": X_train[:, cat_id]
    })
    
trainset = tf.data.Dataset.from_tensor_slices((trainset_input, trainset_outputs))
trainset = trainset.shuffle(1024).batch(32, drop_remainder=True)

In [61]:
# Define autoencoder path and create dir if it doesn't exist.
heae_path = os.path.join("tensorflow_bank", "ADULT_autoencoder")
if not os.path.exists(heae_path):
    os.makedirs(heae_path)

# Define constants.
EPOCHS = 50             # epochs to train the autoencoder
HIDDEN_DIM = 128         # hidden dimension of the autoencoder
LATENT_DIM = 20          # define latent dimension

# Define output dimensions.
OUTPUT_DIMS = [len(numerical_ids)]
OUTPUT_DIMS += [len(bank.category_map[cat_id]) for cat_id in categorical_ids]

# Define the heterogeneous auto-encoder.
heae = HeAE(encoder=ADULTEncoder(hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM),
            decoder=ADULTDecoder(hidden_dim=HIDDEN_DIM, output_dims=OUTPUT_DIMS))

# Define loss functions.
he_loss = [keras.losses.MeanSquaredError()]
he_loss_weights = [1.]

# Add categorical losses.
for i in range(len(categorical_names)):
    he_loss.append(keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    he_loss_weights.append(1./len(categorical_names))

# Define metrics.
metrics = {}
for i, cat_name in enumerate(categorical_names):
    metrics.update({f"output_{i+2}": keras.metrics.SparseCategoricalAccuracy()})
    
# Compile model.
heae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
             loss=he_loss,
             loss_weights=he_loss_weights,
             metrics=metrics)

if len(os.listdir(heae_path)) == 0:
    # Fit and save autoencoder.
    heae.fit(trainset, epochs=EPOCHS)
    heae.save(heae_path, save_format="tf")
else:
    # Load the model.
    heae = keras.models.load_model(heae_path, compile=False)

In [62]:
# 输出encoder的全部权重（包括隐藏层）
for i, weight in enumerate(heae.encoder.weights):
    print(f"参数{i}: {weight.name}, 形状: {weight.shape}")
    print(weight.numpy())  # 打印具体数值

参数0: adult_encoder/dense_2/kernel:0, 形状: (49, 128)
[[ 0.16955987 -0.02945541  0.00368213 ... -0.00764013 -0.0503218
  -0.08211114]
 [ 0.00210219  0.01771733 -0.05446194 ...  0.01902988  0.22840506
   0.02035353]
 [-0.03494316 -0.01455933 -0.03574897 ... -0.00426898 -0.1830821
   0.11857364]
 ...
 [ 0.06157698  0.09224897  0.15947539 ...  0.11723     0.04373055
   0.15644035]
 [ 0.1731051   0.06974736  0.04557892 ...  0.06204503  0.06142191
  -0.04512652]
 [ 0.19792186  0.12175795  0.19045521 ...  0.03161228  0.13781269
   0.08231993]]
参数1: adult_encoder/dense_2/bias:0, 形状: (128,)
[ 0.07128768  0.01407799  0.07354799  0.02693616  0.02506502  0.02519572
 -0.02094448 -0.00422953  0.09502146  0.09082495 -0.03308912  0.04188995
  0.06604271  0.00718864 -0.03178789 -0.01368643  0.0624559  -0.01332409
 -0.00674581 -0.05733289  0.04470127  0.01302228  0.03336751  0.00172157
  0.03690324  0.04314944 -0.02711393  0.03699955  0.03870979 -0.01512121
  0.08398003  0.02924956 -0.05123693  0.00955863

In [63]:
# Define constants
COEFF_SPARSITY = 0.5               # sparisty coefficient
COEFF_CONSISTENCY = 0.5            # consisteny coefficient
TRAIN_STEPS = 10000              # number of training steps -> consider increasing the number of steps
BATCH_SIZE = 100                   # batch size

In [64]:
import wandb


class RewardCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        
        if (step + update) % 100 != 0:
            return
        
        # get the counterfactual and target
        Y_t = sample["Y_t"]
        X_cf = model.params["decoder_inv_preprocessor"](sample["X_cf"])
        
        # get prediction label
        Y_m_cf = predictor(X_cf)
        
        # compute reward
        reward = np.mean(model.params["reward_func"](Y_m_cf, Y_t))
        wandb.log({"reward": reward})

class LossCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        # Log training losses.
        if (step + update) % 100 == 0:
            wandb.log(losses)

class TablesCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        # Log every 1000 steps
        if step % 1000 != 0:
            return
        
        # Define number of samples to be displayed.
        NUM_SAMPLES = 5
        
        X = heae_inv_preprocessor(sample["X"][:NUM_SAMPLES])        # input instance
        X_cf = heae_inv_preprocessor(sample["X_cf"][:NUM_SAMPLES])  # counterfactual
        
        Y_m = np.argmax(sample["Y_m"][:NUM_SAMPLES], axis=1).astype(int).reshape(-1, 1) # input labels
        Y_t = np.argmax(sample["Y_t"][:NUM_SAMPLES], axis=1).astype(int).reshape(-1, 1) # target labels
        Y_m_cf = np.argmax(predictor(X_cf), axis=1).astype(int).reshape(-1, 1)          # counterfactual labels
        
        # Define feature names and category map for input.
        feature_names = bank.feature_names + ["Label"]
        category_map = deepcopy(bank.category_map)
        category_map.update({feature_names.index("Label"): bank.target_names})
        
        # Construct input array.
        inputs = np.concatenate([X, Y_m], axis=1)
        inputs = pd.DataFrame(apply_category_mapping(inputs, category_map),
                              columns=feature_names)
        
        # Define feature names and category map for counterfactual output.
        feature_names += ["y"]
        category_map.update({feature_names.index("y"): bank.target_names})
        
        # Construct output array.
        outputs = np.concatenate([X_cf, Y_m_cf, Y_t], axis=1)
        outputs = pd.DataFrame(apply_category_mapping(outputs, category_map),
                               columns=feature_names)
        
        # Log table.
        wandb.log({
            "Input": wandb.Table(dataframe=inputs),
            "Output": wandb.Table(dataframe=outputs)
        })

In [65]:
# Define immutable features.
immutable_features = ['sex']

# Define ranges. This means that the `Age` feature can not decrease.
ranges = {'age': [0.0, 1.0]}

# Initialize wandb.
wandb_project = "Adult Census Counterfactual with Reinforcement Learning"
wandb.init(project=wandb_project)

explainer = CounterfactualRLTabular(predictor=predictor,
                                    encoder=heae.encoder,
                                    decoder=heae.decoder,
                                    latent_dim=LATENT_DIM,
                                    encoder_preprocessor=heae_preprocessor,
                                    decoder_inv_preprocessor=heae_inv_preprocessor,
                                    coeff_sparsity=COEFF_SPARSITY,
                                    coeff_consistency=COEFF_CONSISTENCY,
                                    category_map=bank.category_map,
                                    feature_names=bank.feature_names,
                                    immutable_features=immutable_features,
                                    ranges=ranges,
                                    train_steps=TRAIN_STEPS,
                                    batch_size=BATCH_SIZE,
                                    backend="tensorflow",
                                    callbacks=[LossCallback(), RewardCallback(), TablesCallback()])

actor_loss,▇█▃▂▁▂▃▂▂▂
consistency_loss,█▄▂▂▂▁▁▁▁▁
critic_loss,▇█▆▆▄▃▃▂▁▁
reward,▁▁▇▇███▇██
sparsity_cat_loss,█▇▇▅▄▄▅▄▄▁
sparsity_num_loss,█▄▄▃▃▁▂▁▁▁
actor_loss,-0.98851
consistency_loss,0.1016
critic_loss,0.04133
reward,0.96
sparsity_cat_loss,0.38


In [ ]:


# Fit the explainer.
explainer = explainer.fit(X=X_train)

# Close wandb.
wandb.finish()

actor_loss,▇█▃▂▁▂▃▂▂▂
consistency_loss,█▄▂▂▂▁▁▁▁▁
critic_loss,▇█▆▆▄▃▃▂▁▁
reward,▁▁▇▇███▇██
sparsity_cat_loss,█▇▇▅▄▄▅▄▄▁
sparsity_num_loss,█▄▄▃▃▁▂▁▁▁
actor_loss,-0.98851
consistency_loss,0.1016
critic_loss,0.04133
reward,0.96
sparsity_cat_loss,0.38


 10%|█         | 1000/10000 [01:01<09:15, 16.20it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\a1227\\AppData\\Local\\Temp\\tmpn0qz8frrwandb-media\\p8kgmdmh.table.json'

In [ ]:
# Generate counterfactual instances.
sample_num = 5543
X = X_train[sample_num].reshape(1, -1)
Y_t = np.array([1 - Y_train[sample_num]])
C = [{
'age': [0, 20]       # 限定 age 在 0~70
}]
explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=4, batch_size=10)

1it [00:00,  6.80it/s]


In [ ]:
X_cf = explanation.data['cf']['X']
predictor(X_cf).argmax(axis=1)

array([1, 1, 1, 1], dtype=int64)

In [ ]:
# Concat labels to the original instances.
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

# Concat labels to the counterfactual instances.
cf = np.concatenate(
    [X_cf, explanation.data['cf']['class']],
    axis=1
)

# Define new feature names and category map by including the label.
feature_names = bank.feature_names + ["y"]
category_map = deepcopy(bank.category_map)
category_map.update({feature_names.index("y"): bank.target_names})

# Replace label encodings with strings.
orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names
)

In [ ]:
orig_pd.head(n=5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,47,management,married,secondary,no,177,yes,no,cellular,14,may,95,3,unknown,0


In [ ]:

cf_pd.head(n=5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,poutcome,y
0,47,management,married,secondary,no,516,yes,no,cellular,14,mar,83,3,success,1
1,47,management,married,secondary,no,636,yes,no,cellular,14,mar,94,3,success,1
2,47,management,married,secondary,no,652,yes,no,cellular,14,jun,105,3,success,1
3,47,management,married,tertiary,no,453,yes,no,cellular,14,aug,283,3,failure,1


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*has feature names.*')
import time

cfs = []


start = time.time()
for i, X_test_i in enumerate(X_test[:100]):
    X = X_test_i.reshape(1, -1)
    Y_t = 1 - predictor(X).argmax(axis=1)
    C = [{
    'age': [0, 20]       # 限定 age 在 0~70
    }]
    explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=4, batch_size=10)
    
    # Concat labels to the original instances.
    orig = np.concatenate(
        [explanation.data['orig']['X'], explanation.data['orig']['class']],
        axis=1
    )

    # Concat labels to the counterfactual instances.
    cf = np.concatenate(
        [explanation.data['cf']['X'], explanation.data['cf']['class']],
        axis=1
    )

    # Define new feature names and category map by including the label.
    feature_names = bank.feature_names + ["y"]
    category_map = deepcopy(bank.category_map)
    category_map.update({feature_names.index("y"): bank.target_names})

    # Replace label encodings with strings.
    orig_pd = pd.DataFrame(
        apply_category_mapping(orig, category_map),
        columns=feature_names
    )

    cf_pd = pd.DataFrame(
        apply_category_mapping(cf, category_map),
        columns=feature_names
    )
    cfs.append((
        orig_pd, cf_pd
    ))
end = time.time()

time_Alibi_cfrl = (end - start)/100

1it [00:00, 10.33it/s]
1it [00:00, 10.30it/s]
1it [00:00, 11.43it/s]
1it [00:00, 10.27it/s]
1it [00:00,  9.80it/s]
1it [00:00,  9.61it/s]
1it [00:00, 11.06it/s]
1it [00:00, 11.32it/s]
1it [00:00, 10.58it/s]
1it [00:00, 10.12it/s]
1it [00:00, 10.99it/s]
1it [00:00, 11.23it/s]
1it [00:00, 11.31it/s]
1it [00:00, 11.91it/s]
1it [00:00, 10.69it/s]
1it [00:00,  9.74it/s]
1it [00:00, 10.30it/s]
1it [00:00,  9.56it/s]
1it [00:00,  9.75it/s]
1it [00:00, 10.31it/s]
1it [00:00, 10.73it/s]
1it [00:00,  8.37it/s]
1it [00:00,  9.16it/s]
1it [00:00,  9.38it/s]
1it [00:00,  9.83it/s]
1it [00:00, 10.77it/s]
1it [00:00, 11.91it/s]
1it [00:00, 11.23it/s]
1it [00:00, 10.25it/s]
1it [00:00, 10.66it/s]
1it [00:00, 10.47it/s]
1it [00:00, 10.39it/s]
1it [00:00, 11.34it/s]
1it [00:00, 11.13it/s]
1it [00:00, 10.81it/s]
1it [00:00, 10.41it/s]
1it [00:00,  7.82it/s]
1it [00:00,  5.93it/s]
1it [00:00,  6.00it/s]
1it [00:00,  6.28it/s]
1it [00:00,  6.10it/s]
1it [00:00,  6.96it/s]
1it [00:00,  6.91it/s]
1it [00:00,

In [ ]:
# 2. 映射
def custom_label_encoder(feature_names, df):
    arr = []
    for _, row in df.iterrows():
        encoded_row = []
        for i, f in enumerate(feature_names):
            if i in category_map:
                val = row[f]
                idx = category_map[i].index(val)
                encoded_row.append(idx)
            else:
                encoded_row.append(float(row[f]))
        arr.append(encoded_row)
    # 返回二维 array，类型 float32
    return np.array(arr)[:, :-1]


In [ ]:
valid_count = 0
total = 0
for X_org, X_cfs in cfs:
    # 预测原始类别（X_org必须二维）
    y_true = model.predict(preprocessor.transform(custom_label_encoder(feature_names, X_org)), 
                           verbose=0).argmax(axis=1)[0]
    # 预测反事实类别（批量）
    y_cfs = model.predict(preprocessor.transform(custom_label_encoder(feature_names, X_cfs)), 
                          verbose=0).argmax(axis=1)
    # 统计类别发生变化的样本数
    valid_count += np.sum(y_cfs != y_true)
    total += len(y_cfs)
valid_alibi_cfrl = (valid_count / total if total > 0 else 0)

In [ ]:
from XAI_metrics import calc_sparsity, calc_continuous_proximity, \
    calc_categorical_proximity, calc_manifold_distance, calc_cf_num


sparsity_alibi_cfrl = calc_sparsity(cfs, categorical_features)
con_proximity_alibi_cfrl = calc_continuous_proximity(cfs, numerical_names)
cat_proximity_alibi_cfrl = calc_categorical_proximity(cfs, categorical_features)
manifold_alibi_cfrl = calc_manifold_distance(cfs, df, categorical_features)
cf_num_alibi_cfrl = calc_cf_num(cfs)

In [ ]:
results_alibi_cfrl = {
    "method": ["Alibi_CFRL"],
    "Avg Time(s)": [time_Alibi_cfrl],
    "Validity": [valid_alibi_cfrl],
    "Sparsity": [sparsity_alibi_cfrl],
    "Proximity_con": [con_proximity_alibi_cfrl],
    "Proximity_cat": [cat_proximity_alibi_cfrl],
    "Manifold": [manifold_alibi_cfrl],
    "Avg CF count": [cf_num_alibi_cfrl]
}

df_results_alibi_cfrl = pd.DataFrame(results_alibi_cfrl)
df_results_alibi_cfrl = df_results_alibi_cfrl.round(2)

,method,Avg Time(s),Validity,Sparsity,Proximity_con,Proximity_cat,Manifold,Avg CF count
0,Alibi_CFRL,0.260404,1.0,0.391281,0.805354,0.188056,21.747942,4.0


In [ ]:
df_results_alibi_cfrl.to_csv('./results/Alibi_CFRL_result_bank.csv', index=False)

: 